<a href="https://colab.research.google.com/github/trahinhasan/Liver-Disease-Prediction-Using-Deep-Learning/blob/main/liver_diseases_prediction_ae_dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
column_names = [
    'Age',
    'Gender',
    'Total_Bilirubin',
    'Direct_Bilirubin',
    'Alkaline_Phosphotase',
    'Alamine_Aminotransferase',
    'Aspartate_Aminotransferase',
    'Total_Proteins',
    'Albumin',
    'Albumin_and_Globulin_Ratio',
    'Dataset'
]

data = pd.read_csv("/content/sample_data/Indian Liver Patient Dataset (ILPD).csv", header=None, names=column_names)
data.head()

In [ ]:
data['Albumin_and_Globulin_Ratio'].fillna(
    data['Albumin_and_Globulin_Ratio'].median(), inplace=True
)

le = LabelEncoder()
data['Gender'] = le.fit_transform(data['Gender'])

X = data.drop('Dataset', axis=1)
y = data['Dataset'].map({1: 1, 2: 0})

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

input_dim = X_train.shape[1]

In [ ]:
# Autoencoder
input_layer = Input(shape=(input_dim,))
encoded = Dense(8, activation='relu')(input_layer)
decoded = Dense(input_dim, activation='linear')(encoded)

autoencoder = Model(input_layer, decoded)
encoder = Model(input_layer, encoded)

autoencoder.compile(
    optimizer='adam',
    loss='mse'
)

In [ ]:
autoencoder.fit(
    X_train, X_train,
    epochs=100,
    batch_size=16,
    validation_split=0.2,
    callbacks=[EarlyStopping(patience=10, restore_best_weights=True)],
    verbose=1
)

In [ ]:
X_train_enc = encoder.predict(X_train)
X_test_enc = encoder.predict(X_test)

In [ ]:
# DNN Classifier
classifier = tf.keras.Sequential([
    Dense(32, activation='relu', input_shape=(X_train_enc.shape[1],)),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

classifier.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
classifier.fit(
    X_train_enc, y_train,
    epochs=100,
    batch_size=16,
    validation_split=0.2,
    callbacks=[EarlyStopping(patience=10, restore_best_weights=True)],
    verbose=1
)

In [ ]:
y_prob = classifier.predict(X_test_enc).ravel()
y_pred = (y_prob > 0.5).astype(int)

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_prob))